In [2]:
from fastaiv07.learner import *
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
ratings = np.random.randint(5, size=(5,5))
ratings

array([[3, 2, 0, 4, 3],
       [4, 1, 1, 1, 1],
       [1, 2, 1, 4, 2],
       [0, 2, 4, 4, 3],
       [1, 1, 3, 3, 3]])

In [4]:
def get_emb(ni,nf):
    e = torch.nn.Embedding(ni, nf)
    e.weight.data.uniform_(-0.01,0.01)
    return e

class EmbeddingDotBias(torch.nn.Module):
    def __init__(self, n_users, n_movies, n_factors = 3):
        super().__init__()
        (self.u, self.m, self.ub, self.mb) = [get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors), (n_users,1), (n_movies,1)
        ]]
        
    def forward(self, cats, conts):
        users,movies = cats[:,0],cats[:,1]
        um = (self.u(users)* self.m(movies)).sum(1)
        res = um + self.ub(users).squeeze() + self.mb(movies).squeeze()
        res = F.sigmoid(res) * (max_rating-min_rating) + min_rating
        return res.view(-1, 1)

In [5]:
wd=2e-4
model = EmbeddingDotBias(5, 5)
opt = torch.optim.SGD(model.parameters(), 1e-1, weight_decay=wd, momentum=0.9)

In [ ]:
fit(model, data, 3, opt, F.mse_loss)